In [1]:
import sqlite3
conn = sqlite3.connect('data/WDIProd.sqlite')


In [124]:
def getMaxMinForData(row):
    return r'SELECT max("{}") as maxvalue, min("{}") as minValue, avg("{}") as avgvalue FROM "data"'.format(row, row, row)

In [125]:
getMaxMinForData("Test")

'SELECT max("Test") as maxvalue, min("Test") as minValue, avg("Test") as avgvalue FROM "data"'

In [117]:
class DataSelector:
    def __init__(self, yearDelta, metrics):
        self.yearDelta = yearDelta
        self.metrics = metrics
    
class QueryConstructor:
    def __init__(self, dataSelector):
        self.dataSelector = dataSelector
        
    def formartTableName(self, table, row):
        return r'"{}"."{}"'.format(table,row)
        
    def formartName(self, table):
        return r'"{}"'.format(table)
        
    def constructForYear(self, year):
        output = ""
        
        #SELECT PART FIRST
        output += r'SELECT '
        
        for selectedData in self.dataSelector.metrics:
            output += self.formartTableName("time1",selectedData)
            output += " as "
            output += self.formartName("p."+selectedData)
            output += " , "
        
        output += " "
        
        #Country Name as N row
        output += self.formartTableName("origins","CountryCode")
        output += ", "
        
        #T+0 GDP as N+1 row
        output += self.formartTableName("time2","NY.GDP.PCAP.PP.KD")
        output += " as gdpf ,"
        
        #T+yY GDP as N+2 row
        output += self.formartTableName("time1","NY.GDP.PCAP.PP.KD")
        output += " as gdpp "
            
        year2 = year
        year1 = year - self.dataSelector.yearDelta
        
        data = r' from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origins" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origins"."CountryCode" AND "time1"."Year" == {} LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origins"."CountryCode" AND "time2"."Year" == {} WHERE gdpf is not NULL AND gdpp is not NULL'.format(year1,year2)
        
        output += data
        
        return output

In [ ]:
# SELECT "origin1"."CountryCode", "time1"."EN.ATM.NOXE.KT.CE" as "time1.EN.ATM.NOXE.KT.CE", "time2"."EN.ATM.NOXE.KT.CE" as "time2.EN.ATM.NOXE.KT.CE" from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origin1" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origin1"."CountryCode" AND "time1"."Year" == 1990 LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origin1"."CountryCode" AND "time2"."Year" == 1991

In [149]:
ds = DataSelector(5,["EN.MAM.THRD.NO","BX.TRF.PWKR.DT.GD.ZS"])

In [150]:
qc = QueryConstructor(ds)

In [120]:
qc.constructForYear(2018)

'SELECT "time1"."BX.TRF.PWKR.DT.GD.ZS" as "p.BX.TRF.PWKR.DT.GD.ZS" , "time1"."EN.MAM.THRD.NO" as "p.EN.MAM.THRD.NO" ,  "origins"."CountryCode", "time2"."NY.GDP.PCAP.PP.KD" as gdpf ,"time1"."NY.GDP.PCAP.PP.KD" as gdpp  from (SELECT DISTINCT "origin"."CountryCode" from "data" as "origin") as "origins" LEFT JOIN "data" as "time1" ON "time1"."CountryCode" == "origins"."CountryCode" AND "time1"."Year" == 2013 LEFT JOIN "data" as "time2" ON "time2"."CountryCode" == "origins"."CountryCode" AND "time2"."Year" == 2018 WHERE gdpf is not NULL AND gdpp is not NULL'

In [58]:
list(conn.execute(qc.constructForYear(2018)))

[(1.68845930330232, None, 'AFG', 2190.24032132044, 2264.3175752325),
 (0.0267993914860461, None, 'AGO', 6933.50930820608, 8140.53120789329),
 (10.0330801692658, None, 'ALB', 13601.3033805754, 11597.9632735891),
 (2.07731781241171, None, 'ARB', 14620.2610718972, 14136.9464230217),
 (None, None, 'ARE', 66968.2698548892, 59921.7348033071),
 (0.0969166746867399, None, 'ARG', 22745.9037844103, 24424.3504112585),
 (19.7113750362956, None, 'ARM', 12714.9581875586, 10691.3099550604),
 (1.78825882234923, None, 'ATG', 21115.7982687308, 17641.5425333279),
 (0.15159377815888, None, 'AUS', 49575.981131221, 47192.0573890772),
 (0.758368508471961, None, 'AUT', 55687.1892771302, 53289.3783085002),
 (2.33692601835619, None, 'AZE', 14209.6494059309, 14652.7042457179),
 (1.98396848206385, None, 'BDI', 761.524193384446, 877.274160261081),
 (2.24197236441209, None, 'BEL', 51245.9962234656, 48243.0201797298),
 (1.98756670280948, None, 'BEN', 3160.7770925266, 2876.90563418335),
 (2.29442584641875, None, 'BFA

In [155]:
def Normalize(meanv, minmaxdiffv):
    """
    :param a: Array
    :return: Array
    """
    mean = meanv
    maxmindiff = minmaxdiffv

    def apply(v):
        if v is None:
            return 0
        return (v - mean) / maxmindiff

    def apply_inversed(v):
        return v * maxmindiff + mean

    return apply, apply_inversed


class DataFromSelector:
    def __init__(self, dataSelector, yearmin, years, conn):
        self.dataSelector = dataSelector
        self.yearmin = yearmin
        self.years = years # The window of years
        
        self.qc = QueryConstructor(self.dataSelector)
        self.conn = conn
        
        
        normalF = []
        for mat in self.dataSelector.metrics:
            nf = self.getNormalizeFunction(mat)[0]
            normalF.append(nf)
        
        self.normalF = normalF
        
    def getNormalizeFunction(self,metric):
        sql = getMaxMinForData(metric)
        
        maxmin = self.conn.execute(sql).fetchone()
        
        maxv = maxmin[0]
        minv = maxmin[1]
        
        avgv = maxmin[2]
        
        return Normalize(avgv,maxv-minv)
        
    def getProsperityIndex(self, gdp1, gdp2):
        if gdp2-gdp1 > 0:
            return pow((gdp2-gdp1)/gdp1,(1/self.years))
        else:
            return - pow((gdp1-gdp2)/gdp1,(1/self.years))
    
    def getXfromRow(self, row):
        rawData = row[:-3]
        normalF = self.normalF
        
        data = []
        for item in range(len(rawData)):
                data.append(normalF[item](rawData[item]))
        return data
    
    def getYfromRow(self, row):
        return self.getProsperityIndex(row[-2],row[-1])
        
        #return (row[-2],row[-1])
    
    def constructOneYear(self, year, X, Y):
        sql = self.qc.constructForYear(year)
        res = self.conn.execute(sql)
        for data in res:
            X.append(self.getXfromRow(data))
            Y.append(self.getYfromRow(data))
    
    def constructAll(self):
        X = []
        Y = []
        
        for ye in range(self.yearmin, self.yearmin+self.years):
            self.constructOneYear(ye, X,Y)
        
        return (X,Y)

In [156]:
dfs = DataFromSelector(ds,2000,15,conn)

In [159]:
len(dfs.constructAll()[1])

3435


Size of the economy (GDP per capita) (http://wdi.worldbank.org/table/WV.1) NY.GDP.PCAP.PP.KD
Poverty rates at international poverty lines (http://wdi.worldbank.org/table/1.2) SI.POV.LMIC.GP SI.POV.GINI
Distribution of income or consumption (http://wdi.worldbank.org/table/1.3) SI.DST.10TH.10 SI.DST.05TH.20 SI.DST.FRST.10 SI.DST.FRST.20 SI.DST.50MD
Labor force structure (http://wdi.worldbank.org/table/2.2)  SL.TLF.0714.ZS SL.EMP.1524.SP.ZS  SL.TLF.ADVN.ZS SL.TLF.BASC.ZS SL.TLF.INTM.ZS
Employment by sector (http://wdi.worldbank.org/table/2.3) SL.IND.EMPL.ZS SL.ISV.IFRM.ZS  SL.EMP.WORK.ZS SL.AGR.EMPL.ZS 
Unemployment (http://wdi.worldbank.org/table/2.5) SL.UEM.1524.ZS SL.UEM.TOTL.ZS
Health systems (http://wdi.worldbank.org/table/2.12)
Energy dependency, efficiency and carbon dioxide emissions (http://wdi.worldbank.org/table/3.8)
Structure of merchandise exports (http://wdi.worldbank.org/table/4.4)
Structure of merchandise imports (http://wdi.worldbank.org/table/4.5)
Structure of service exports (http://wdi.worldbank.org/table/4.6)
Structure of service imports (http://wdi.worldbank.org/table/4.7)
Business environment: Doing Business indicators (http://wdi.worldbank.org/table/5.3)
Financial access, stability and efficiency (http://wdi.worldbank.org/table/5.5)
Tax policies (http://wdi.worldbank.org/table/5.6)

In [160]:
ds = DataSelector(5,["NY.GDP.PCAP.PP.KD","SI.POV.LMIC.GP","SI.DST.10TH.10","SI.DST.50MD"])

In [163]:
dfs = DataFromSelector(ds,1970,45,conn)

In [164]:
len(dfs.constructAll()[1])

4480

In [ ]:
class CombinedDataSelector:
    def __init__(self,daArray):
        self.daArray = daArray
        